# Digit Recgnizer

# Setup

In [2]:
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row

import org.apache.spark.ml.feature._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification._

import org.apache.spark.ml.linalg._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._


import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp
import java.time.temporal.ChronoUnit

In [3]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<!-- To left align the HTML components in Markdown -->

In [4]:
val NUM_CORES = 4
val NUM_PARTITIONS = 4

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/

Name: Compile Error
Message: <console>:71: error: stable identifier required, but this.$line7$read.spark.implicits found.
       import spark.implicits._
                    ^

StackTrace: 

In [5]:
val configMap = spark.conf.getAll.foreach(println)

Waiting for a Spark session to start...

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,10.138.111.209)
(spark.eventLog.enabled,true)
(spark.driver.port,36869)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://10.138.111.209:36869/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-2b364b40-7f3e-430d-9553-d54c5d78e599/repl-03a7e26f-3e7c-4606-bf3f-7dd89ec1460b)
(spark.app.name,Apache Toree)
(spark.driver.memory,2g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://10.138.111.209:4041)
(spark.org.apache.hadoop.yarn.server.webprox

configMap: Unit = ()


## Constant

In [10]:
val PROTOCOL="hdfs://"
val DATA_DIR="/data/kaggle/mnist"
val RESULT_DIR="."

val COVERAGE:Double = 0.95

PROTOCOL = hdfs://
DATA_DIR = /data/kaggle/mnist
RESULT_DIR = .
COVERAGE = 0.95


0.95

# MNIST Dataset

In [12]:
val train = spark.read.format("csv")
    .option("header", true)
    .option("inferSchema", true)
    .load(PROTOCOL + DATA_DIR + "/train.csv.gz")

train = [label: int, pixel0: int ... 783 more fields]


[label: int, pixel0: int ... 783 more fields]

In [13]:
val numFeatures = train.columns.size - 1  // Remove "label" column
val numSamples = train.count

numFeatures = 784
numSamples = 42000


42000

# Dataframe for ML training

Use R formula transformation to create label and features columns for Spark ML.

In [29]:
val formula = "label ~ ."
val rformula = new RFormula()
  .setFormula(formula)
  .fit(train)

val trainDataDF = rformula
    .transform(train)
    .select("label", "features")

formula = label ~ .
rformula = RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...


RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...

In [30]:
trainDataDF.printSchema`

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
trainDataDF.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(784,[132,133,134...|
|    0|(784,[122,123,124...|
|    1|(784,[124,125,126...|
|    4|(784,[146,147,148...|
|    0|(784,[121,122,123...|
|    0|(784,[124,125,126...|
|    7|(784,[202,203,204...|
|    3|(784,[177,178,179...|
|    5|(784,[153,154,155...|
|    3|(784,[119,120,121...|
+-----+--------------------+
only showing top 10 rows



# SVD

In [16]:
import org.apache.spark.sql.Row
def rowToVector(r: Row): org.apache.spark.mllib.linalg.Vector = {
    val v = r.getAs[org.apache.spark.ml.linalg.Vector](0)
    org.apache.spark.mllib.linalg.Vectors.fromML(v)
}

rowToVector: (r: org.apache.spark.sql.Row)org.apache.spark.mllib.linalg.Vector


In [33]:
val trainDataMatrix = new RowMatrix(
    trainDataDF.select("features").rdd.map(rowToVector)
)

trainDataMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@48a89061


org.apache.spark.mllib.linalg.distributed.RowMatrix@48a89061

In [34]:
val svd: SingularValueDecomposition[RowMatrix, Matrix] = trainDataMatrix.computeSVD(numFeatures, computeU = true)
val U: RowMatrix = svd.U  // The U factor is a RowMatrix.
val s: Vector = svd.s     // The singular values are stored in a local dense vector.
val V: Matrix = svd.V     // The V factor is a local dense matrix.

svd = 


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@2f47a2e8,[323503.703778161,109669.14717327854,101621.48745300347,93843.81264344015,87209.07876311651,78809.72055197941,72567.49168291429,64828.82470555653,64416.68658534496,58308.861751418466,55406.27915475731,54750.55160735113,49609.08790700611,49550.52462799281,47953.954194702324,46331.72758655017,44275.238616400704,43359.10907006242,41448.81945675147,40775.83066771042,39324.43779854911,38274.73752844696,37316.21577405969,36320.82031107616,35793.95790153957,34803.45941347596,34247.123321015686,33487.606203318144,32683.269877895247,31472.91710587883,30818.6...


In [35]:
val singluarValues = s.toDense.values

singluarValues = Array(323503.703778161, 109669.14717327854, 101621.48745300347, 93843.81264344015, 87209.07876311651, 78809.72055197941, 72567.49168291429, 64828.82470555653, 64416.68658534496, 58308.861751418466, 55406.27915475731, 54750.55160735113, 49609.08790700611, 49550.52462799281, 47953.954194702324, 46331.72758655017, 44275.238616400704, 43359.10907006242, 41448.81945675147, 40775.83066771042, 39324.43779854911, 38274.73752844696, 37316.21577405969, 36320.82031107616, 35793.95790153957, 34803.45941347596, 34247.123321015686, 33487.606203318144, 32683.269877895247, 31472.91710587883, 30818.644127794658, 30355.606099707322, 29404.347846111687, 29255.758990244067, 28531.531896135904, 27953.24406512744, 27100.342265174775, 26516.189399254694, 26189.552346276807, 259...


Array(323503.703778161, 109669.14717327854, 101621.48745300347, 93843.81264344015, 87209.07876311651, 78809.72055197941, 72567.49168291429, 64828.82470555653, 64416.68658534496, 58308.861751418466, 55406.27915475731, 54750.55160735113, 49609.08790700611, 49550.52462799281, 47953.954194702324, 46331.72758655017, 44275.238616400704, 43359.10907006242, 41448.81945675147, 40775.83066771042, 39324.43779854911, 38274.73752844696, 37316.21577405969, 36320.82031107616, 35793.95790153957, 34803.45941347596, 34247.123321015686, 33487.606203318144, 32683.269877895247, 31472.91710587883, 30818.644127794658, 30355.606099707322, 29404.347846111687, 29255.758990244067, 28531.531896135904, 27953.24406512744, 27100.342265174775, 26516.189399254694, 26189.552346276807, 259...

## Number of principal components (k) to cover COVERAGE

In [21]:
val windowSpec = Window
  .orderBy(desc("singluar_value"))
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)
  
val windowSpecAll = Window
  .orderBy(desc("singluar_value"))
  .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

val coverageDF = sc.parallelize(singluarValues).toDF("singluar_value")
    .withColumn(
        "id",
        row_number().over(windowSpec)
    )
    .select("id", "singluar_value")
    .withColumn(
        "eignvalue", 
        pow(col("singluar_value"), 2) / lit(numSamples -1)
    )
    .withColumn(
        "cumulativeSum", 
        sum(col("eignvalue")).over(windowSpec)
    )
    .withColumn(
        "coverage", 
        col("cumulativeSum") / last(col("cumulativeSum")).over(windowSpecAll)
    )
coverageDF.show(5)

+---+------------------+------------------+------------------+-------------------+
| id|    singluar_value|         eignvalue|     cumulativeSum|           coverage|
+---+------------------+------------------+------------------+-------------------+
|  1|  323503.703778161| 2491836.623685996| 2491836.623685996|0.43500390900934977|
|  2|109669.14717327854| 286371.6241271037|   2778208.2478131|0.48499626193511053|
|  3|101621.48745300347|245885.06183863763|3024093.3096517376| 0.5279208108602296|
|  4| 93843.81264344015|209687.40140139285|  3233780.71105313| 0.5645262762477199|
|  5| 87209.07876311651|181085.82153649992|  3414866.53258963| 0.5961387180449769|
+---+------------------+------------------+------------------+-------------------+
only showing top 5 rows



windowSpec = org.apache.spark.sql.expressions.WindowSpec@68c286f5
windowSpecAll = org.apache.spark.sql.expressions.WindowSpec@364c16ba
coverageDF = [id: int, singluar_value: double ... 3 more fields]


[id: int, singluar_value: double ... 3 more fields]

In [28]:
val kDF = coverageDF.where(col("coverage") >= lit(COVERAGE)).limit(1)
kDF.show

val k = kDF.select("id").collect()(0).getInt(0)

+---+------------------+------------------+-----------------+------------------+
| id|    singluar_value|         eignvalue|    cumulativeSum|          coverage|
+---+------------------+------------------+-----------------+------------------+
|102|11675.026832748754|3245.4642145147122|5443028.651091638|0.9501982263075994|
+---+------------------+------------------+-----------------+------------------+



kDF = [id: int, singluar_value: double ... 3 more fields]
k = 102


102

# PCA

In [54]:
val pca = new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(k)
  .fit(trainDataDF)

pca = pca_755c79044108


pca_755c79044108

In [55]:
pca.explainParams()

inputCol: input column name (current: features)
k: the number of principal components (> 0) (current: 102)
outputCol: output column name (default: pca_755c79044108__output, current: pcaFeatures)


In [56]:
pca.explainedVariance

[0.09748937689497461,0.07160266275027234,0.06145903355957282,0.05379301996327574,0.04894262134042076,0.04303213992139485,0.03277050763580546,0.028921031736972334,0.02766902346628119,0.02348871029531937,0.02099325425271383,0.020590011597844095,0.017025534993579613,0.01692787021349282,0.015811264063787012,0.01483239617649628,0.013196878946996466,0.012827270808166676,0.01187976141356012,0.011527547329596514,0.010721912242319678,0.010151993021548662,0.00964902259415799,0.009128460679541317,0.008876408591128955,0.008387663079557076,0.008118558545110089,0.007774057474595946,0.007406351163183782,0.006866614888087311,0.006579822105222276,0.0063879861144869815,0.005993670159253401,0.005889134101342426,0.005643351784057687,0.005409670481475439,0.00...

In [57]:
pca.explainedVariance.values.sum

0.9169591926906049

In [60]:
val pcaTrainDataDF = pca.transform(trainDataDF)

pcaTrainDataDF = [label: int, features: vector ... 1 more field]


[label: int, features: vector ... 1 more field]

In [61]:
pcaTrainDataDF.printSchema
pcaTrainDataDF.show(10)

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- pcaFeatures: vector (nullable = true)

+-----+--------------------+--------------------+
|label|            features|         pcaFeatures|
+-----+--------------------+--------------------+
|    1|(784,[132,133,134...|[103.738813757989...|
|    0|(784,[122,123,124...|[2466.78627830941...|
|    1|(784,[124,125,126...|[-121.55984060477...|
|    4|(784,[146,147,148...|[599.578991071955...|
|    0|(784,[121,122,123...|[2689.04430947599...|
|    0|(784,[124,125,126...|[1253.08650413365...|
|    7|(784,[202,203,204...|[93.0114290617944...|
|    3|(784,[177,178,179...|[650.952778816167...|
|    5|(784,[153,154,155...|[1115.56395904828...|
|    3|(784,[119,120,121...|[1062.72668192117...|
+-----+--------------------+--------------------+
only showing top 10 rows



# Random Forest

In [75]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
val rf = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol("pcaFeatures")
  .setFeatureSubsetStrategy("auto")

rf.explainParams()

rf = rfc_54f91692ea43


lastException: Throwable = null
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all...


## Performance evaluator

In [64]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("accuracy")

evaluator = mcEval_7be655db0fab


mcEval_7be655db0fab

## Grid search parameters

In [76]:
import org.apache.spark.ml.tuning.ParamGridBuilder
val params = new ParamGridBuilder()
    .addGrid(rf.maxDepth, Array(3, 5))
    .addGrid(rf.numTrees, Array(10, 20))
    .addGrid(rf.subsamplingRate, Array(0.1, 0.5))
  .build()

params = 


Array({
	rfc_54f91692ea43-maxDepth: 3,
	rfc_54f91692ea43-numTrees: 10,
	rfc_54f91692ea43-subsamplingRate: 0.1
}, {
	rfc_54f91692ea43-maxDepth: 3,
	rfc_54f91692ea43-numTrees: 10,
	rfc_54f91692ea43-subsamplingRate: 0.5
}, {
	rfc_54f91692ea43-maxDepth: 3,
	rfc_54f91692ea43-numTrees: 20,
	rfc_54f91692ea43-subsamplingRate: 0.1
}, {
	rfc_54f91692ea43-maxDepth: 3,
	rfc_54f91692ea43-numTrees: 20,
	rfc_54f91692ea43-subsamplingRate: 0.5
}, {
	rfc_54f91692ea43-maxDepth: 5,
	rfc_54f91692ea43-numTrees: 10,
	rfc_54f91692ea43-subsamplingRate: 0.1
}, {
	rfc_54f91692ea43-maxDepth: 5,
	rfc_54f91692ea43-numTrees: 10,
	rfc_54f91692ea43-subsamplingRate: 0.5
}, {
	rfc_54f91692ea43-maxDepth: 5,
	rfc_54f91692...


## Cross Validation

In [77]:
import org.apache.spark.ml.tuning.{TrainValidationSplit, TrainValidationSplitModel}
val tvs = new TrainValidationSplit()
  .setTrainRatio(0.75) // also the default.
  .setEstimatorParamMaps(params)
  .setEstimator(rf)
  .setEvaluator(evaluator)

tvs = tvs_55ae6576be9e


tvs_55ae6576be9e

In [ ]:
val tvsFitted = tvs.fit(pcaTrainDataDF)

In [70]:
tvsFitted.write.overwrite().save("/data/model/rf.mdl")

In [ ]:
tvsFitterd.transform